# Learning From the Data, Homework 8

In [75]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold

## Primal versus Dual Problem

### Problem 1

Recall that $N$ is the size of the data set and $d$ is the dimensionality of the input space. The original formulation of the hard-margin SVM (minimize $\frac{1}{2}\mathbf{w}^T\mathbf{w}$ subject to the inequality constraints), without going through the Lagrangian dual problem, is...

#### Solution

We are minimizing $\mathbf{w}$ with constraints on $\mathbf{w}$ and $b$. So, the number of variables will be the number of weights (length of $\mathbf{w}$) plus one additional variable for $b$. The number of weights corresponds to the dimensionality $d$. Thus, the correct answer is a quadratic programming problem with $d+1$ variables.

Answer: [**d**] a quadratic programming problem with $d+1$ variables

## SVM with Soft Margins

In the rest of the problems of this homework set, we apply soft-margin SVM to handwritten digits from the processed US Postal Service Code data set. Download the data (exracted features of intensity and symmetry) for training and testing:

http://www.amlbook.com/data/zip/features.train

http://www.amlbook.com/data/zip/features.test

(the format of each row is: **digit intensity symmetry**). We will train two types of binary classifiers; one-versus-one (one digit is class +1 and another digit is class -1, with the rest of the digits disregarded), and one-versus-all (one digit is class +1 and the rest of the digits are class -1).

Implement SVM with soft margin on the above zip-code data set by solving:

$\min_\alpha \frac{1}{2}\sum_{n=1}^N\sum_{m=1}^N\alpha_{n}\alpha_{m}y_{n}y_{m}K(\mathbf{x}_n\mathbf{x}_m)-\sum_{n=1}^N\alpha_n$

$\text{s.t.} \sum_{n=1}^{N}y_n\alpha_n = 0$

$0 \leq \alpha_n \leq C \quad n=1,...,N$

When evaluating $E_\text{in}$ and $E_\text{out}$ of the resulting classifier, use binary classification error.

## Polynomial Kernels

Consdier the polynomial kernel $K(\mathbf{x}_n, \mathbf{x}_m) = (1 + \mathbf{x}_n^T\mathbf{x}_m)^Q$, where $Q$ is the degree of the polynomial.

### Problem 2

With $C=0.01$ and $Q=2$, which of the following classifiers has the **highest** $E_\text{in}$?

#### Solution

In [72]:
train = pd.read_table("http://www.amlbook.com/data/zip/features.train", 
                      delim_whitespace=True, header=None).values

def one_versus(d, num, num2=None):
    d = d.copy()
    if num2 is not None:
        d = d[(d[:,0] == num) | (d[:,0] == num2),:]
    d[d[:,0] == num, 0] = -1
    d[d[:,0] != -1, 0] = -2
    d[d[:,0] == -1, 0] = 1
    d[d[:,0] == -2, 0] = 0
    return d[:,1:], d[:,0]

print("{:<6} {:<10} {:<10}".format("Digit", "E_in", "SVs"))
for i in range(10):
    X, Y = one_versus(train, i)
    clf = SVC(kernel='poly', degree=2, gamma=1.0, coef0=1.0, C=0.01)
    clf.fit(X, Y)
    print("{:<6} {:<10.8f} {:<10}"
          .format(i, 1 - clf.score(X, Y), np.sum(clf.n_support_)))

Digit  E_in       SVs       
0      0.10588397 2179      
1      0.01440132 386       
2      0.10026060 1970      
3      0.09024825 1950      
4      0.08942532 1856      
5      0.07625840 1585      
6      0.09107118 1893      
7      0.08846523 1704      
8      0.07433823 1776      
9      0.08832808 1978      


Answer: [**a**] 0 versus all

### Problem 3

With $C=0.01$ and $Q=2$, which of the following classifiers has the lowest $E_\text{in}$?

#### Solution

Answer: [**a**] 1 versus all


### Probem 4

Comparing the two selected classifiers from Problems 2 and 3, which of the following values is the closest to the difference between the number of support vectors of these two classifiers?

#### Solution

In [61]:
2179-386

1793

Answer: [**c**] 1800

#### Problem 5

Consider the 1 versus 5 clssifier with $Q=2$ and $C \in \{0.001, 0.01, 0.1, 1\}$. Which of the following statements is correct? Going up or down means strictly so.

#### Solution

In [73]:
test = pd.read_table("http://www.amlbook.com/data/zip/features.test", 
                      delim_whitespace=True, header=None).values

X, Y = one_versus(train, 1, 5)
X_test, Y_test = one_versus(test, 1, 5)

print("{:<6} {:<10} {:<10} {:<10}"
      .format("C", "E_in", "E_out", "SVs"))
for i in [0.001, 0.01, 0.1, 1.0]:
    clf = SVC(kernel='poly', degree=2, gamma=1.0, coef0=1.0, C=i)
    clf.fit(X, Y)
    e_in = 1 - clf.score(X, Y)
    e_out = 1 - clf.score(X_test, Y_test)
    print("{:<6} {:<10.8f} {:<10.8f} {:<10}"
          .format(i, e_in, e_out, np.sum(clf.n_support_)))
    

C      E_in       E_out      SVs       
0.001  0.00448430 0.01650943 76        
0.01   0.00448430 0.01886792 34        
0.1    0.00448430 0.01886792 24        
1.0    0.00320307 0.01886792 24        


Answer: [**d**] Maximum $C$ achieves the lowest $E_\text{in}$.

### Problem 6

In the 1 versus 5 classifier, comparing $Q = 2$ with $Q = 5$, which of the following statements is correct?

#### Solution

In [74]:
print("{:<2} {:<6} {:<10} {:<10} {:<10}"
      .format("Q", "C", "E_in", "E_out", "SVs"))
for i in [0.0001, 0.001, 0.01, 1.0]:
    for q in [2, 5]:
        clf = SVC(kernel='poly', degree=q, gamma=1.0, coef0=1.0, C=i)
        clf.fit(X, Y)
        e_in = 1 - clf.score(X, Y)
        e_out = 1 - clf.score(X_test, Y_test)
        print("{:<2} {:<6} {:<10.8f} {:<10.8f} {:<10}"
              .format(q, i, e_in, e_out, np.sum(clf.n_support_)))

Q  C      E_in       E_out      SVs       
2  0.0001 0.00896861 0.01650943 236       
5  0.0001 0.00448430 0.01886792 26        
2  0.001  0.00448430 0.01650943 76        
5  0.001  0.00448430 0.02122642 25        
2  0.01   0.00448430 0.01886792 34        
5  0.01   0.00384369 0.02122642 23        
2  1.0    0.00320307 0.01886792 24        
5  1.0    0.00320307 0.02122642 21        


Answer: [**b**] When $C=0.001$, the number of support vectors is lower than $Q=5$.

## Cross Validation

In the next two problems, we will experiment with 10-fold cross validation for the polynomial kernel. Because $E_\text{cv}$ is a random variable that depends on the random partition of the data, we will try 100 runs with different partitions and base our answer on how many runs lead to a particular choice.

### Problem 7

Consdier the 1 versus 5 classifier with $Q=2$. We use $E_\text{cv}$ to select $C \in \{0.0001, 0.001, 0.01, 0.1, 1\}$. If there is a tie in $E_\text{cv}$, select the smaller $C$. Within the 100 rnadom runs, which of the following statements is correct?

#### Solution

In [119]:
def run_experiment():
    k_fold = KFold(n_splits=10, shuffle=True)
    X, Y = one_versus(train, 1, 5)
    E_cv = np.zeros(5)
    for idx, c in enumerate([0.0001, 0.001, 0.01, 0.1, 1]):
        clf = SVC(kernel='poly', degree=2, gamma=1.0, coef0=1.0, C=c)
        accuracy = [clf.fit(X[train_idx], Y[train_idx])
                        .score(X[test_idx], Y[test_idx])
                    for train_idx, test_idx 
                    in k_fold.split(Y)]
        E_cv[idx] = np.mean(1 - np.array(accuracy))
    return E_cv, np.min(np.where(E_cv == np.min(E_cv)))

counts = {'0':0, '1':0, '2':0, '3':0, '4':0}

total_E = 0.0

for i in range(100):
    E_cv, idx = run_experiment()
    total_E += E_cv[1]
    counts[str(idx)] += 1

print(counts, total_E/100)
    

{'0': 0, '1': 30, '2': 25, '3': 24, '4': 21} 0.00474077249714


Answer: [**b**] $C = 0.001$ is selected most often.

### Problem 8

Again, consider the 1 versus 5 classifier with $Q=2$. For the winning selection in the previous problem, the average value of $E_\text{cv}$ over the 100 runs is closest to

#### Solution

See code and output from Problem 7.

Answer: [**c**] 0.005

## RBF Kernel

Consider the radial basis function (RBF) kernel $K(\mathbf{x}_n, \mathbf{x}_m) = \exp (-||\mathbf{x}_n - \mathbf{x}_m||^2)$ in the soft-margin SVM approach. Focus on the 1 versus 5 classifier.

### Problem 9

Which of the following values of $C$ results in the lowest $E_\text{in}$?

#### Solution

In [128]:
X, Y = one_versus(train, 1, 5)
X_test, Y_test = one_versus(test, 1, 5)

print("{:<6} {:<10} {:<10} {:<10}"
      .format("C", "E_in", "E_out", "SVs"))
for i in [0.01, 1, 100, 1e4, 1e6]:
    clf = SVC(kernel='rbf', gamma=1.0, C=i)
    clf.fit(X, Y)
    e_in = 1 - clf.score(X, Y)
    e_out = 1 - clf.score(X_test, Y_test)
    print("{:<6.0E} {:<10.8f} {:<10.8f} {:<10}"
          .format(i, e_in, e_out, np.sum(clf.n_support_)))

C      E_in       E_out      SVs       
1E-02  0.00384369 0.02358491 406       
1E+00  0.00448430 0.02122642 31        
1E+02  0.00320307 0.01886792 22        
1E+04  0.00256246 0.02358491 19        
1E+06  0.00064061 0.02358491 17        


Answer: [**e**] $10^6$

### Problem 10

Which of the following values of C results in the lowest $E_\text{out}$?

#### Solution

See code and output from Problem 9.

Answer: [**c**] $C=100$